In [39]:
import numpy as np
from tqdm import tqdm

In [40]:
patterns = []
with open("day14.txt") as f:
    lines = f.readlines()
current_pattern = []
for line in lines:
    if len(line) > 3:
        line = line[:-1]
        current_pattern.append(line)
    else:
        patterns.append(current_pattern)
        current_pattern = []
patterns.append(current_pattern)
patterns[0]
#beware - new line must be at input

['.O....O.OOO..O.#.#.###.#O....OO..OO.O...O...O###..#.#....#.#....##..O.O.O##.......O.#.O.......##O#O.',
 '..#.#.#......O........O#OOOO#.O.#.#.#..O.#O...#..O...#..O.O#......#O...##O...O.#...O.O...##O.O.O...#',
 '.......O.OO..O.##.#..O.#...O#.#.......O.O.##.......#.O..#.......O.....#...O...#.#OO......#....#.....',
 '....#.....O#.O..O..#..#OO.#OO....#.......O..O.O...#.....#.O.....O..O.O....#..#O..O.#.O....O.......#O',
 '.........O.O....O.#O..#..........#O#......###.#..OO...#.O#O.O.O#....##...O.O..#.O..OO.#.OO...O#.OOO.',
 '.#O..#.O....#O...O#.........O#O..#O..#O.O.OO..O..#...#O...O..OOOO.....O..O.........O..#O##......O##.',
 '..O#.O.....#.O..O.#.O......O#O.#..#.O.O..#..O.O#..O..#O.....##.#....#..OO.#.........OO....#...O....O',
 '.O.O#..O.###....O#....O...#....O.O....O.O.O......O.O###OO.#.O...O..O..O.....O.#.O#..OO..O#.OO#OO.#OO',
 'OO...#O#......#####OOO.O###...........O#....#.#.O.O.##...O#.O...........OO.#...#......O.O....#..O..#',
 '.O..OO....#.#.##O.O#..OO....#...O..O...O.O###.##O....

In [41]:
pattern = patterns[0]
x_dim = len(pattern[0])
y_dim = len(pattern)
x_dim, y_dim

(100, 100)

In [42]:
symbols = np.zeros((y_dim, x_dim), dtype="str")

In [43]:
numbers = []
for y, row in enumerate(pattern):
    for x, val in enumerate(row):
        symbols[y, x] = val

In [44]:
def pretty_print(matrix):
    print(str(matrix).replace(' ','').replace("'","").replace('[','').replace(']',''))

In [45]:
pretty_print(symbols)

.O....#O.
..#.....#
.........
...
..#.....O
.O.......
.......OO


In [46]:
from functools import cache


@cache
def calculate_weight(round_rocks, distance_from_south):
    if round_rocks:
        return distance_from_south + calculate_weight(round_rocks-1, distance_from_south-1) # could be replaced by formula
    return 0

In [47]:
calculate_weight(2,2)

3

In [48]:
y_dim = len(pattern)
x_dim = len(pattern[0])

In [49]:
pretty_print(symbols)

.O....#O.
..#.....#
.........
...
..#.....O
.O.......
.......OO


In [50]:
input_symbols = symbols.copy()

In [51]:
pretty_print(input_symbols)

.O....#O.
..#.....#
.........
...
..#.....O
.O.......
.......OO


In [52]:
def tilt_north(new_symbols, x_dim, y_dim):
    for x in range(x_dim):
        current_small_rocks = 0
        for distance_from_south in range(y_dim):
            current_symbol = new_symbols[y_dim-distance_from_south-1, x]
            #print(current_symbol)
            if current_symbol == "O":
                current_small_rocks += 1
                new_symbols[y_dim-distance_from_south-1, x] = "."
            elif current_symbol == "#":
                for new_symbol_position in range(y_dim-distance_from_south, y_dim-distance_from_south+current_small_rocks):
                    new_symbols[new_symbol_position, x] = "O"
                current_small_rocks = 0
        if current_small_rocks:
            for new_symbol_position in range(current_small_rocks):
                        new_symbols[new_symbol_position, x] = "O"
    return new_symbols

In [53]:
def till_south(new_symbols, x_dim, y_dim):
    return np.flip(tilt_north(np.flip(new_symbols, axis=0), x_dim, y_dim), axis=0)

In [54]:
def till_west(new_symbols, x_dim, y_dim):
    for y in range(y_dim):
        current_small_rocks = 0
        for distance_from_west in range(x_dim):
            current_symbol = new_symbols[y, x_dim-distance_from_west-1]
            #print(current_symbol)
            if current_symbol == "O":
                current_small_rocks += 1
                new_symbols[y, x_dim-distance_from_west-1 ] = "."
            elif current_symbol == "#":
                for new_symbol_position in range(x_dim-distance_from_west, x_dim-distance_from_west+current_small_rocks):
                    new_symbols[y, new_symbol_position] = "O"
                current_small_rocks = 0
        if current_small_rocks:
            for new_symbol_position in range(current_small_rocks):
                        new_symbols[y, new_symbol_position] = "O"
    return new_symbols

In [55]:
def till_east(new_symbols, x_dim, y_dim):
    return np.flip(till_west(np.flip(new_symbols, axis=1), x_dim, y_dim), axis=1)

In [56]:
new_symbols = input_symbols.copy()
pretty_print(new_symbols)

.O....#O.
..#.....#
.........
...
..#.....O
.O.......
.......OO


In [57]:
north = tilt_north(new_symbols, x_dim, y_dim)
pretty_print(north)

OO....#O.
O.#...O.#
O.O.....O
...
..#....O.
.........
.........


In [58]:
west = till_west(north.copy(), x_dim, y_dim)
pretty_print(west)

OOO...#O.
O.#.....#
OOO...O..
...
..#......
.........
.........


In [59]:
south = till_south(west.copy(), x_dim, y_dim)
pretty_print(south)

..O...#..
.O#.....#
.O.....O.
...
O.#......
O........
OO......O


In [60]:
east = till_east(south.copy(), x_dim, y_dim)
pretty_print(east)

......#..
.O#....O#
........O
...
.O#....OO
........O
.OO...OOO


In [61]:
visited = set()

In [62]:
# find cycle length - seems the config will repeat after some time
def find_repeat_length(new_symbols, x_dim, y_dim):   
    cycle = 0 
    while True:
        key = tuple(new_symbols.flatten())
        if key in visited:
            return cycle
        visited.add(key)
        north = tilt_north(new_symbols, x_dim, y_dim)
        west = till_west(north, x_dim, y_dim)
        south = till_south(west, x_dim, y_dim)
        east = till_east(south, x_dim, y_dim)
        new_symbols = east
        cycle += 1

In [63]:
def calculate_load(new_symbols, x_dim, y_dim):
    total_weight = 0
    for x in range(x_dim):
        for distance_from_south in range(y_dim):
            current_symbol = new_symbols[y_dim-distance_from_south-1, x]
            #print(current_symbol)
            if current_symbol == "O":
                total_weight += distance_from_south+1
    return total_weight

In [64]:
def do_cycles(new_symbols, x_dim, y_dim, cycles=1):
    results = []    
    for _ in range(cycles):
        north = tilt_north(new_symbols, x_dim, y_dim)
        west = till_west(north, x_dim, y_dim)
        south = till_south(west, x_dim, y_dim)
        east = till_east(south, x_dim, y_dim)
        new_symbols = east
        weight = calculate_load(new_symbols, x_dim, y_dim)
        results.append(weight)
    return results

In [65]:
cycles = 1000000000

In [66]:
new_symbols = input_symbols.copy()
repeat_cycle = find_repeat_length(new_symbols, x_dim, y_dim)
repeat_cycle

151

In [67]:
#equivalent_cycles = repeat_cycle + (cycles % repeat_cycle)
equivalent_cycles = 1000

In [68]:
#new_symbols = input_symbols.copy()
#new_symbols = do_cycles(new_symbols, x_dim, y_dim, 10000)
#pretty_print(new_symbols)

In [69]:

new_symbols = input_symbols.copy()
results = do_cycles(new_symbols, x_dim, y_dim, 350)
results
    
#pretty_print(new_symbols)

[100410,
 100331,
 100112,
 99906,
 99813,
 99754,
 99654,
 99581,
 99475,
 99325,
 99249,
 99180,
 99184,
 99169,
 99160,
 99064,
 99009,
 98925,
 98920,
 98867,
 98826,
 98758,
 98690,
 98603,
 98538,
 98455,
 98433,
 98372,
 98340,
 98302,
 98262,
 98191,
 98149,
 98080,
 98003,
 97929,
 97844,
 97717,
 97608,
 97463,
 97331,
 97175,
 97039,
 96875,
 96745,
 96602,
 96483,
 96324,
 96225,
 96081,
 95952,
 95822,
 95725,
 95602,
 95537,
 95461,
 95389,
 95311,
 95268,
 95244,
 95231,
 95211,
 95172,
 95140,
 95121,
 95062,
 95008,
 94951,
 94895,
 94844,
 94808,
 94768,
 94716,
 94671,
 94627,
 94588,
 94563,
 94526,
 94510,
 94487,
 94491,
 94502,
 94511,
 94535,
 94560,
 94591,
 94618,
 94620,
 94615,
 94585,
 94565,
 94537,
 94510,
 94493,
 94487,
 94494,
 94517,
 94536,
 94565,
 94587,
 94610,
 94626,
 94616,
 94590,
 94561,
 94529,
 94516,
 94494,
 94492,
 94490,
 94509,
 94542,
 94566,
 94592,
 94606,
 94618,
 94622,
 94591,
 94566,
 94525,
 94508,
 94500,
 94493,
 94495,
 9450

# you can see there is a cycle (271 to 341)

In [70]:
results[271]

94597

In [71]:
results[341]

94597

In [72]:
start = results[:271]

In [73]:
toapp = results[271:341]

In [74]:
index = (cycles - len(start)) % len(toapp)

In [76]:
toapp[index-1: index+1]

[94585, 94565]